### TODO:
- Criar função check_double para checar se após uma alocação, alguém está com turno dobrado.
    - Caso sim, alocar folga novamente
- Repetir lógica do solve até todos os shifts estarem alocados ou encontrar um erro
- Criar lógica para lidar com soft constraints

In [1]:
from enum import Enum
from typing import List
from dateutil.relativedelta import relativedelta
from datetime import datetime, date
import pandas as pd
import numpy as np
from copy import deepcopy, copy
import bisect # Used for fast searching of first x greater than K in a list

In [2]:
#Criar dataframe contendo posições e nomes
morning_workers = ['CHEF1', 'COZ1', 'COZ2', 'ASG1', 'ASG2', 'AUX1', 'CONF1', 'PIZ1', 'GARD1', 'MASSA1']
evening_workers = ['CHEF2', 'COZ3', 'COZ4', 'ASG3', 'ASG4', 'AUX2', 'AUXPIZ1', 'PIZ2', 'GARD2', 'MASSA2']
workers = morning_workers + evening_workers
positions = [x[:-1] for x in workers]
df = pd.DataFrame({'position': positions, 'worker': workers})
#Update caso particular
df.loc[df['worker'] == 'AUXPIZ1', 'position'] = 'PIZ'
df = pd.concat([df, pd.DataFrame([['CONF','AUXPIZ1']])], axis = 1)

In [3]:
# These definitions will help set constraints in an intuitive manner by the user
MONDAY = 0
TUESDAY = 1
WEDNESDAY = 2
THURSDAY = 3
FRIDAY = 4
SATURDAY = 5
SUNDAY = 6

In [33]:
class CalendarDay():
    """
    A CalendarDay represents a day of the month.
    Attrs:
        - Date
        - Day of the week
        - Number of Shifts
        - Week of the year
    """
   
    
    def __init__(self, date: date):
        self.date = date
        self.date_alias = self.date.strftime('%a%d')
        self.week_day = self.date.weekday()
        self.week = self.date.strftime('%W')
        self.next_day = None
    
    def __repr__(self):
        return f"{self.date_alias}"
        
class Position():
    """
    Even though Position has a pretty simple implementation, it's easier if it has its own class from a Data Model and user interaction perspective.
    """
    def __init__(self, name: str):
        self.name = name.upper()
        
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return self.name

class Shift():
    """
    A Shift needs a set number of workers per position (slots) to be filled.
    """
    def __init__(self, cal_day: CalendarDay, no: int, positions: List[Position], no_workers_per_position: dict, candidates):
        self.cal_day = cal_day
        self.shift_name = str(self.cal_day.date_alias) + ' S' +str(no).zfill(2)
        self.week_day = cal_day.week_day
        self.week = cal_day.week
        self.no = no
        self.workers = []
        self.slots_to_fill = no_workers_per_position
        self.filled_slots = {}
        self.candidates = candidates
        for position in positions:
            #self.slots_to_fill[str(position)] = no_workers_per_position
            self.filled_slots[str(position)] = set()
        
    def update(self, slots_to_fill, candidates):
        """
        Allows the user to specify slots_to_fill using a dictionary with keys: positions 
        and values: number of workers for that position; and the candidates dictionary with
        keys: position, values: set of worker names 
        """
        self.slots_to_fill = slots_to_fill
        self.candidates = candidates
        for position in self.slots_to_fill.keys():
            self.filled_slots[str(position)] = set()
    
    def __repr__(self):
        return self.shift_name
    

class Worker():
    def __init__(self, name: str, position: Position, availability: List[Shift] = []):
        self.name = name
        self.position = position
        self.availability = []
        self.no_shifts = 0
        self.shift_preference = 100
        self.days_off = []
        
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return self.name



class Schedule():
    """
    Class which contains the calendar and the solver (AC3).
    Initializes the days and shifts. Sets requirements for each shift in terms of workers and positions. Executes solver.
    Users should pass the positions names and requirements, as well as the worker list.
    attrs:
        - start_date: 'YYYYMMDD'
        - end_date: 'YYYYMMDD'
        - number_of_shifts: number of shifts within a day
        - number_of_positions: number of distinct roles that should be allocated within each shift
        - calendar_days
        
    methods:
        - add_contraint
        - remove_constraint
        - solve
    """
    
    next_month = (date.today() + relativedelta(months=1))
    first_day = next_month + relativedelta(days=-next_month.day + 1)
    last_day = first_day + relativedelta(months=1, days=-1)
    
    def __init__(self, 
                 no_workers_per_position: dict,
                 start_date: str = first_day,
                 end_date: str = last_day, 
                 no_shifts_per_day: int = 2, 
                 df: pd.DataFrame = pd.DataFrame({'position': [], 'worker': []}),
                 
                ):
        self.start_date = start_date
        self.end_date = end_date
        self.constraints = {}
        self.df = df
        self.no_workers_per_position = no_workers_per_position
        self.positions, self.workers = self.get_workers_and_positions(df)
        self.no_positions = len(positions)
        self.calendar_days = [CalendarDay(x) for x in pd.date_range(start_date, end_date, freq='1d')]
        for i in range(0, len(self.calendar_days)):
            if i < len(self.calendar_days) - 1:
                self.calendar_days[i].next_day = self.calendar_days[i + 1]
        #candidates_per_position should be combined with worker availability
        self.candidates_per_position = dict(df.groupby('position')['worker'].unique().apply(set))
        #self.candidates_per_position = {Position(k):v for k,v in self.candidates_per_position.items()}
        #Initialize shifts
        self.shifts = [] 
        self.substitutes = {'CONF': 'AUXPIZ1'}
        self.worker_allocation = {}
        for cal_day in self.calendar_days:
            for shift_no in range(1, no_shifts_per_day + 1):
                self.shifts.append(Shift(cal_day,shift_no, self.positions, self.no_workers_per_position, self.candidates_per_position))
        #Initialize worker availability
        for worker in self.workers:
            worker.availability = copy(self.shifts)
            self.worker_allocation[worker.name] = 0
        
    
    def get_workers_and_positions(self, df):
        """
        Takes a pandas DataFrame containing name of the worker and position
        Returns all the positions and workers.
        """
        positions = [Position(x) for x in df['position'].unique()]
        workers = []
        for row in df.itertuples():
            worker = Worker(row[2], Position(row[1]))
            workers.append(worker)
        return positions, workers
    
    
    
    def add_schedule_constraint(self, kind='days_off_per_week', **kwargs):
        """
            - kind: 
                worker_to_shiftno 
                    - soft constraint - try to enforce after hard constraints
                    - worker can only work on shift n 
                    - kwargs: worker_name, shift_no
                days_off_per_week 
                    - hard constraint
                    - kwargs: no_days
                days_off_per_month 
                    - hard constraint
                    - kwargs: no_days
                days_off_per_weekday_per_month 
                    - hard constraint
                    - kwargs: no_days, weekday                    
                consecutive_days_off 
                    - soft constraint
                    - kwargs: no_days
                day_off_after_doubling 
                    - hard_constraint
                    - kwargs: flag
        """
        if kind == 'days_off_per_week':
            try:
                no_days = kwargs['no_days']
                self.constraints[kind] = {'no_days': no_days}
            except KeyError:
                print(f'You must specify no_days when kind is {kind}!')
        elif kind == 'days_off_per_month':
            try:
                no_days = kwargs['no_days']
                self.constraints[kind] = {'no_days': no_days}
            except KeyError:
                print(f'You must specify no_days when kind is {kind}!')
        elif kind == 'days_off_per_weekday_per_month':
            try:
                no_days = kwargs['no_days']
                week_day = kwargs['week_day']
                self.constraints[kind] = {'no_days': no_days, 'week_day': week_day}
            except KeyError:
                print(f'You must specify no_days and week_day when kind is {kind}!')
        elif kind == 'day_off_after_doubling':
            self.constraints[kind] = True
        elif kind == 'consecutive_days_off':
            try:
                no_days = kwargs['no_days']
                self.constraints[kind] = {'no_days': no_days}
            except KeyError:
                print(f'You must specify no_days when kind is {kind}!')
        #Implement consecutive_days_off and worker_to_shiftno
    
    def update_number_of_shifts_for_particular_day(self):
        pass
        
    def remove_worker_availability_by_day(self, worker: Worker, day: CalendarDay):
        shifts = self.get_shifts_by_calendar_day(day)
        #print(shifts)
        worker.days_off.append(day.date)
        for shift in shifts:
            worker.availability.remove(shift)
        
     # ----------------- Helper querying functions ----------------------- #
    def get_shifts_by_number(self, no):
        return [x for x in self.shifts if x.no == no]
    
    def get_shifts_by_calendar_day(self, cal_day):
        return [x for x in self.shifts if x.cal_day == cal_day]
    
    def get_shifts_by_day_of_the_week(self, week_day):
        return [x for x in self.shifts if x.week_day == week_day]
    
    def get_shift_by_name(self, name):
        return [x for x in self.shifts if x.shift_name == name]
    
    def get_calday_by_day_of_the_week(self, week_day):
        return [x for x in self.calendar_days if x.week_day == week_day]
    
    def get_calday_by_date(self, date):
        return [x for x in self.calendar_days if x.date == date][0]
    
    def get_workers_by_calday_availability(self, calday):
        """Returns worker names"""
        shifts = self.get_shifts_by_calendar_day(calday)
        workers = set()
        for shift in shifts:
            for worker in self.workers:
                if shift in worker.availability:
                    workers.add(worker.name)
        return workers
    
    def get_workers_by_position(self, position):
        return [x for x in self.workers if x.position.name == position.name]
    
    def get_worker_by_name(self, name):
        return [x for x in self.workers if x.name == name][0]
        
    # ------------------------------------------------------------------- #
    
    def allocate(self, days: List[CalendarDay]):
        """Allocates available workers in the specified days"""
        for day in days:
            print(f"Day: {day}")
            shifts = self.get_shifts_by_calendar_day(day)
            #print(shifts)
            available_workers = self.get_workers_by_calday_availability(day)
            print(f'Available workers: {available_workers}')
            for shift in shifts:
                #print(shift)
                for position in shift.slots_to_fill.keys():
                    n_workers = shift.slots_to_fill[position]
                    #print(shift.candidates.keys())
                    #print(f"Position: {position}")
                    position_candidates = set(shift.candidates[position])
                    #print(f"Position candidates: {position_candidates}")
                    new_candidates = list(available_workers.intersection(position_candidates))
                    new_candidates = sorted(new_candidates, key=lambda x: self.worker_allocation.get(x), reverse=True)
                    #new_candidates = sorted(new_candidates, key=lambda x: self.worker_allocation.get(x), reverse=True)
                    #print(f"New candidates: {new_candidates}")
                    while len(shift.filled_slots[position]) < n_workers:
                        if len(new_candidates) > 0:
                            min_allocation = self.worker_allocation.get(new_candidates[-1])
                            ideal_worker_to_allocate = []
                            alternative_worker_to_allocate = []
                            for worker in new_candidates:
                                if self.worker_allocation.get(worker) == min_allocation:
                                    worker_instance = self.get_worker_by_name(worker)
                                    if worker_instance.shift_preference == shift.no:
                                        ideal_worker_to_allocate.append(worker)
                                    else:
                                        alternative_worker_to_allocate.append(worker)
                            if len(ideal_worker_to_allocate) > 0:
                                worker_to_allocate = ideal_worker_to_allocate[-1]
                            else:
                                worker_to_allocate = alternative_worker_to_allocate[-1]
                            shift.filled_slots[position].add(worker_to_allocate)
                            self.worker_allocation[worker_to_allocate] += 1
                            new_candidates.remove(worker_to_allocate)
                        else:
                            shift.filled_slots[position].add(self.substitutes[position])
    
    def solve(self, debug=False):
        #Start by looking at days_off_per_weekday_per_month
        if 'days_off_per_weekday_per_month' in self.constraints.keys():
            no_days = self.constraints['days_off_per_weekday_per_month']['no_days']
            week_day = self.constraints['days_off_per_weekday_per_month']['week_day']
            #Get weekdays
            days = self.get_calday_by_day_of_the_week(week_day)
            if debug:
                print('--------------------------------------------')
                print('Days off per weekday')
                print('Days: ', days)
            #Crete auxiliary varibles
            calendario_folga = {}
            folgados = set()
            #Each person has to have at least 1 day off on a sunday per month
            for day in days:
                #shifts = self.get_shifts_by_calendar_day(day)
                if debug:
                    print(f'\nDay:{day}')
                for position in self.positions:
                    if day not in calendario_folga.keys():
                        calendario_folga[day] = {}
                    candidatos_a_folgado = list(set([x for x in self.workers if x.position.name == position.name]) - folgados)
                    if debug:
                        print(f"day:{day}, position:{position}, candidatos_a_folgado: {candidatos_a_folgado}")
                    if len(candidatos_a_folgado) > 0:
                        folgado = np.random.choice(candidatos_a_folgado)
                        if debug:
                            print(f'Folgado escolhido: {folgado}')
                            print(f'Disponibilidade do folgado escolhido: {folgado.availability}')
                        calendario_folga[day][position] = folgado
                        try:
                            self.remove_worker_availability_by_day(folgado, day)
                            folgados.add(folgado)
                            if debug:
                                print(f'Removed availability for worker {folgado} in day {day}')
                        except Exception as e:
                            print(f"Exception {e}")
                            print(folgado, day, folgado.availability)
                            print('\n')
                        
            print('Calendario Folga: ')
            print(calendario_folga)
            #Make sure everyone has a day off on a sunday
            temp = []
            for v in calendario_folga.values():
                for v1 in v.values():
                    temp.append(v1)
            assert(len(set(self.workers) - set(temp)) == 0)
            #First allocation
            print('\n----------------------------------------\n')
            print('First allocation')
            self.allocate(days)
        print('\n----------------------------------------\n')
        
        #Maybe do this after the second day off of the week
        if 'day_off_after_doubling' in self.constraints.keys():
            worker_names = [x.name for x in self.workers]
            #Check days in which employees had two shifts
            doubled = []
            for day in self.calendar_days:
                shifts = self.get_shifts_by_calendar_day(day)
                for worker_name in worker_names:
                    counter = 0
                    for shift in shifts:
                        vals = flat_list = [elem for s in shift.filled_slots.values() for elem in s]
                        if worker_name in vals:
                            counter += 1
                    if counter == 2:
                        doubled.append([day, worker_name])
            print(f"Doubled: {doubled}")
            #Assign day off the next day
            print("Assigning day off after doubling...")
            days = []
            for day, worker_name in doubled:
                next_day = day.next_day
                worker = self.get_worker_by_name(worker_name)
                self.remove_worker_availability_by_day(worker, next_day)
                days.append(next_day)
            #Allocate day off after doubling
            self.allocate(days)
            
        if 'days_off_per_week' in self.constraints.keys():
            no_days = self.constraints['days_off_per_weekday_per_month']['no_days']
            for worker in self.workers:
                # Verify last day off
                worker_name = worker.name
                
                # Define new day off within 7 days
                # Do this until no more days are left in the calendar
                # Assert no more than 7 days have passed since last day off
                pass
            
                
    
    def __repr__(self):
        output_str = 'Schedule for '
        output_str += f'{self.start_date} '
        output_str += f'to {self.end_date}:\n'
        output_str += f'\t- Positions: {" ".join([x.name for x in self.positions])}\n'
        output_str += f'\t- Workers: {" ".join([x.name for x in self.workers])}\n'
        return output_str
    



### Create Generic Schedule

In [34]:
no_workers_per_position = {
    'CHEF': 1,
    'COZ': 2,
    'ASG': 2,
    'AUX': 1,
    'CONF': 1,
    'PIZ': 1,
    'GARD': 1,
    'MASSA': 1
}

In [35]:
#Create generic schedule
schedule = Schedule(df=df, no_workers_per_position=no_workers_per_position)
schedule

Schedule for 2023-09-01 to 2023-09-30:
	- Positions: CHEF COZ ASG AUX CONF PIZ GARD MASSA
	- Workers: CHEF1 COZ1 COZ2 ASG1 ASG2 AUX1 CONF1 PIZ1 GARD1 MASSA1 CHEF2 COZ3 COZ4 ASG3 ASG4 AUX2 AUXPIZ1 PIZ2 GARD2 MASSA2

### Assign Shift preference to workers

In [36]:
for worker in schedule.workers:
    if worker.name in morning_workers:
        worker.shift_preference = 1
    elif worker.name in evening_workers:
        worker.shift_preference = 2

### Add some constraints

In [37]:
#Add some constraints
#Each person has to have 2 days off per week
schedule.add_schedule_constraint(kind='days_off_per_week', no_days=2)
#Each person has to have at least 1 sunday off per month
schedule.add_schedule_constraint(kind='days_off_per_weekday_per_month', no_days=1, week_day=SUNDAY)
#If possible, whoever has the sunday off has monday off as well
schedule.add_schedule_constraint(kind='consecutive_days_off', no_days = 2)
#The same person should only have two shifts on the same day if their next is a day off
schedule.add_schedule_constraint(kind='day_off_after_doubling')
schedule.constraints

{'days_off_per_week': {'no_days': 2},
 'days_off_per_weekday_per_month': {'no_days': 1, 'week_day': 6},
 'consecutive_days_off': {'no_days': 2},
 'day_off_after_doubling': True}

In [38]:
schedule.shifts[5].week

'35'

### Modify specific shifts

##### Update every SUNDAY EVENING

In [39]:
sundays = schedule.get_shifts_by_day_of_the_week(SUNDAY)
sunday_evenings = [x for x in sundays if x.no == 2]
sunday_evenings

[Sun03 S02, Sun10 S02, Sun17 S02, Sun24 S02]

In [40]:
for shift in sunday_evenings:
    shift.update(slots_to_fill = {'CHEF': 1, 'CUSTOM': 2}, 
                 candidates = {'CHEF': set(['CHEF1', 'CHEF2']), 
                               'CUSTOM': set(['PIZ1','PIZ2','AUXPIZ1'])
                              })

##### Inspect shifts

In [41]:
shift = schedule.shifts[0]
print(shift)
print(shift.slots_to_fill)
print('\n')
print(shift.candidates)

Fri01 S01
{'CHEF': 1, 'COZ': 2, 'ASG': 2, 'AUX': 1, 'CONF': 1, 'PIZ': 1, 'GARD': 1, 'MASSA': 1}


{'ASG': {'ASG3', 'ASG4', 'ASG1', 'ASG2'}, 'AUX': {'AUX1', 'AUX2'}, 'CHEF': {'CHEF1', 'CHEF2'}, 'CONF': {'CONF1'}, 'COZ': {'COZ4', 'COZ1', 'COZ2', 'COZ3'}, 'GARD': {'GARD1', 'GARD2'}, 'MASSA': {'MASSA2', 'MASSA1'}, 'PIZ': {'PIZ1', 'PIZ2', 'AUXPIZ1'}}


In [42]:
shift = schedule.shifts[5]

In [43]:
print(shift)
print(shift.slots_to_fill)
print('\n')
print(shift.candidates)

Sun03 S02
{'CHEF': 1, 'CUSTOM': 2}


{'CHEF': {'CHEF1', 'CHEF2'}, 'CUSTOM': {'PIZ1', 'AUXPIZ1', 'PIZ2'}}


### Solver

In [44]:
schedule.solve()

Calendario Folga: 
{Sun03: {CHEF: CHEF1, COZ: COZ4, ASG: ASG4, AUX: AUX2, CONF: CONF1, PIZ: AUXPIZ1, GARD: GARD1, MASSA: MASSA1}, Sun10: {CHEF: CHEF2, COZ: COZ1, ASG: ASG2, AUX: AUX1, PIZ: PIZ1, GARD: GARD2, MASSA: MASSA2}, Sun17: {COZ: COZ3, ASG: ASG3, PIZ: PIZ2}, Sun24: {COZ: COZ2, ASG: ASG1}}

----------------------------------------

First allocation
Day: Sun03
Available workers: {'ASG3', 'PIZ1', 'COZ1', 'MASSA2', 'COZ2', 'PIZ2', 'CHEF2', 'AUX1', 'GARD2', 'ASG1', 'COZ3', 'ASG2'}
Day: Sun10
Available workers: {'GARD1', 'ASG3', 'AUX2', 'AUXPIZ1', 'MASSA1', 'COZ4', 'COZ2', 'CHEF1', 'CONF1', 'PIZ2', 'ASG1', 'COZ3', 'ASG4'}
Day: Sun17
Available workers: {'GARD1', 'AUX2', 'AUXPIZ1', 'PIZ1', 'COZ1', 'MASSA1', 'COZ4', 'ASG4', 'MASSA2', 'CHEF1', 'COZ2', 'CONF1', 'CHEF2', 'AUX1', 'GARD2', 'ASG1', 'ASG2'}
Day: Sun24
Available workers: {'GARD1', 'ASG3', 'AUX2', 'AUXPIZ1', 'PIZ1', 'COZ1', 'MASSA1', 'COZ4', 'ASG4', 'MASSA2', 'CHEF1', 'PIZ2', 'CHEF2', 'AUX1', 'GARD2', 'CONF1', 'COZ3', 'ASG2'}

--

In [45]:
schedule.shifts[5].slots_to_fill

{'CHEF': 1, 'CUSTOM': 2}

In [46]:
schedule.shifts[5]

Sun03 S02

In [47]:
schedule.shifts[4].filled_slots

{'CHEF': {'CHEF2'},
 'COZ': {'COZ1', 'COZ2'},
 'ASG': {'ASG1', 'ASG2'},
 'AUX': {'AUX1'},
 'CONF': {'AUXPIZ1'},
 'PIZ': {'PIZ1'},
 'GARD': {'GARD2'},
 'MASSA': {'MASSA2'}}

In [48]:
schedule.shifts[5].filled_slots

{'CHEF': {'CHEF2'},
 'COZ': set(),
 'ASG': set(),
 'AUX': set(),
 'CONF': set(),
 'PIZ': set(),
 'GARD': set(),
 'MASSA': set(),
 'CUSTOM': {'PIZ1', 'PIZ2'}}

In [49]:
for shift in sundays:
    print('\n--------------------------\n')
    print(f"Shift: {shift}")
    print(f'Slots to fill: {shift.slots_to_fill}')
    print(f'Filled slots: {shift.filled_slots}')


--------------------------

Shift: Sun03 S01
Slots to fill: {'CHEF': 1, 'COZ': 2, 'ASG': 2, 'AUX': 1, 'CONF': 1, 'PIZ': 1, 'GARD': 1, 'MASSA': 1}
Filled slots: {'CHEF': {'CHEF2'}, 'COZ': {'COZ1', 'COZ2'}, 'ASG': {'ASG1', 'ASG2'}, 'AUX': {'AUX1'}, 'CONF': {'AUXPIZ1'}, 'PIZ': {'PIZ1'}, 'GARD': {'GARD2'}, 'MASSA': {'MASSA2'}}

--------------------------

Shift: Sun03 S02
Slots to fill: {'CHEF': 1, 'CUSTOM': 2}
Filled slots: {'CHEF': {'CHEF2'}, 'COZ': set(), 'ASG': set(), 'AUX': set(), 'CONF': set(), 'PIZ': set(), 'GARD': set(), 'MASSA': set(), 'CUSTOM': {'PIZ1', 'PIZ2'}}

--------------------------

Shift: Sun10 S01
Slots to fill: {'CHEF': 1, 'COZ': 2, 'ASG': 2, 'AUX': 1, 'CONF': 1, 'PIZ': 1, 'GARD': 1, 'MASSA': 1}
Filled slots: {'CHEF': {'CHEF1'}, 'COZ': {'COZ4', 'COZ3'}, 'ASG': {'ASG3', 'ASG4'}, 'AUX': {'AUX2'}, 'CONF': {'CONF1'}, 'PIZ': {'AUXPIZ1'}, 'GARD': {'GARD1'}, 'MASSA': {'MASSA1'}}

--------------------------

Shift: Sun10 S02
Slots to fill: {'CHEF': 1, 'CUSTOM': 2}
Filled slot

In [50]:
mondays = schedule.get_shifts_by_day_of_the_week(MONDAY)

In [51]:
for shift in mondays:
    print('\n--------------------------\n')
    print(f"Shift: {shift}")
    print(f'Slots to fill: {shift.slots_to_fill}')
    print(f'Filled slots: {shift.filled_slots}')


--------------------------

Shift: Mon04 S01
Slots to fill: {'CHEF': 1, 'COZ': 2, 'ASG': 2, 'AUX': 1, 'CONF': 1, 'PIZ': 1, 'GARD': 1, 'MASSA': 1}
Filled slots: {'CHEF': {'CHEF1'}, 'COZ': {'COZ1', 'COZ2'}, 'ASG': {'ASG1', 'ASG2'}, 'AUX': {'AUX1'}, 'CONF': {'CONF1'}, 'PIZ': {'PIZ2'}, 'GARD': {'GARD1'}, 'MASSA': {'MASSA1'}}

--------------------------

Shift: Mon04 S02
Slots to fill: {'CHEF': 1, 'COZ': 2, 'ASG': 2, 'AUX': 1, 'CONF': 1, 'PIZ': 1, 'GARD': 1, 'MASSA': 1}
Filled slots: {'CHEF': {'CHEF1'}, 'COZ': {'COZ4', 'COZ3'}, 'ASG': {'ASG3', 'ASG4'}, 'AUX': {'AUX2'}, 'CONF': {'CONF1'}, 'PIZ': {'AUXPIZ1'}, 'GARD': {'GARD2'}, 'MASSA': {'MASSA2'}}

--------------------------

Shift: Mon11 S01
Slots to fill: {'CHEF': 1, 'COZ': 2, 'ASG': 2, 'AUX': 1, 'CONF': 1, 'PIZ': 1, 'GARD': 1, 'MASSA': 1}
Filled slots: {'CHEF': {'CHEF2'}, 'COZ': {'COZ1', 'COZ2'}, 'ASG': {'ASG1', 'ASG2'}, 'AUX': {'AUX1'}, 'CONF': {'CONF1'}, 'PIZ': {'PIZ1'}, 'GARD': {'GARD1'}, 'MASSA': {'MASSA1'}}

------------------------

In [61]:
for x in schedule.calendar_days:
    if x.date == days_off[0]:
        print(x)

Sun10


In [57]:
results = []
for worker in schedule.workers:
    #worker_name = worker.name
    days_off = sorted(worker.days_off)
    cp_days_off = copy(days_off)
    intervals = []
    while len(cp_days_off) > 1: #If worker has been allocated more than 1 day off
        #Compute time between days_off
        interval = cp_days_off[-1] - cp_days_off[-2]
        intervals.append([interval])
        cp_days_off.pop()
    results.append([worker, intervals, days_off])
    #day_availability = [x.cal_day.date for x in worker.availability]
    print(f"Worker: {worker_name},\n  Days off: {days_off},\n Intervals: {intervals}")
    print('-------------------------------------------------------------------------')

Worker: MASSA2,
  Days off: [Timestamp('2023-09-03 00:00:00'), Timestamp('2023-09-11 00:00:00')],
 Intervals: [[Timedelta('8 days 00:00:00')]]
-------------------------------------------------------------------------
Worker: MASSA2,
  Days off: [Timestamp('2023-09-10 00:00:00')],
 Intervals: []
-------------------------------------------------------------------------
Worker: MASSA2,
  Days off: [Timestamp('2023-09-24 00:00:00')],
 Intervals: []
-------------------------------------------------------------------------
Worker: MASSA2,
  Days off: [Timestamp('2023-09-24 00:00:00')],
 Intervals: []
-------------------------------------------------------------------------
Worker: MASSA2,
  Days off: [Timestamp('2023-09-10 00:00:00')],
 Intervals: []
-------------------------------------------------------------------------
Worker: MASSA2,
  Days off: [Timestamp('2023-09-10 00:00:00')],
 Intervals: []
-------------------------------------------------------------------------
Worker: MASSA2,
  

In [66]:
results
for item in results:
    #If only one day off
    if len(item[1]) == 0: #Only one day off so far, therefore no interval to compute
        #Get next available day and turn it into a day_off if there are enough substitutes
        next_x_days = []
        cal_day = schedule.get_calday_by_date(item[0].days_off[0])
        for i in range(6):
            cal_day = cal_day.next_day
            shifts = schedule.get_shifts_by_calendar_day(cal_day)
            for shift in shifts:
                next_x_days.append(shift)
        print(item)
        print(next_x_days)
        print('---------------\n')
    else:
        #Unpack every interval found for each worker and evaluate if it is necessary to introduce a day_off somewhere
        #If so, check if the candidate_day_off has an available substitute
        for nested_item in item[1]:
            print(item[0], nested_item[0].days)
    #for nested_item in item[1]:
    #    print(nested_item)

CHEF1 8
[COZ1, [], [Timestamp('2023-09-10 00:00:00')]]
[Mon11 S01, Mon11 S02, Tue12 S01, Tue12 S02, Wed13 S01, Wed13 S02, Thu14 S01, Thu14 S02, Fri15 S01, Fri15 S02, Sat16 S01, Sat16 S02]
---------------

[COZ2, [], [Timestamp('2023-09-24 00:00:00')]]
[Mon25 S01, Mon25 S02, Tue26 S01, Tue26 S02, Wed27 S01, Wed27 S02, Thu28 S01, Thu28 S02, Fri29 S01, Fri29 S02, Sat30 S01, Sat30 S02]
---------------

[ASG1, [], [Timestamp('2023-09-24 00:00:00')]]
[Mon25 S01, Mon25 S02, Tue26 S01, Tue26 S02, Wed27 S01, Wed27 S02, Thu28 S01, Thu28 S02, Fri29 S01, Fri29 S02, Sat30 S01, Sat30 S02]
---------------

[ASG2, [], [Timestamp('2023-09-10 00:00:00')]]
[Mon11 S01, Mon11 S02, Tue12 S01, Tue12 S02, Wed13 S01, Wed13 S02, Thu14 S01, Thu14 S02, Fri15 S01, Fri15 S02, Sat16 S01, Sat16 S02]
---------------

[AUX1, [], [Timestamp('2023-09-10 00:00:00')]]
[Mon11 S01, Mon11 S02, Tue12 S01, Tue12 S02, Wed13 S01, Wed13 S02, Thu14 S01, Thu14 S02, Fri15 S01, Fri15 S02, Sat16 S01, Sat16 S02]
---------------

[CONF1,